In [6]:
fh=read.csv("four-hundred.csv",sep=",",header=F)
#fh=fh[,-c(2,3,14)]
#length(which(is.na(fh[,1])==T))

In [14]:
k = 5 #Folds

# sample from 1 to k, nrow times (the number of observations in the data)
fh$id <- sample(1:k, nrow(fh), replace = TRUE)
list <- 1:k
cv<-function(f,data){
    X=data[,c(-1)]
y=data[,c(1)]
for(i in 1:ncol(X)){
  X[is.na(X[,i]), i] <- mean(X[,i], na.rm = TRUE)
}

#y[is.na(y)]<-10

    d=cbind(X,y)
   # print((d$id))
  # remove rows with id i from dataframe to create training set
  # select rows with id i to create test set
  trainingset <- subset(d, id %in% list[-f])
  testset <- subset(d, id %in% c(f))
   #print(which(d$id==f))
  #  testset <- d[which(d$id == i),]#subset(d, id %in% list[i])
X.train<-trainingset[,-c(ncol(trainingset),ncol(trainingset)-1)]
X.test<-testset[,-c(ncol(trainingset),ncol(trainingset)-1)]
y.train<-trainingset[,ncol(trainingset)]
y.test<-testset[,ncol(trainingset)]

return(list(X.train=X.train,y.train=y.train,X.test=X.test,y.test=y.test) )   
}

#cv(2,fh)

In [18]:
library("wSVM")


In [16]:
library(gmum.r)
library(SparseM)
library(Matrix)
library(ggplot2)
        
#svm.wt <- SVM(X.train,y.train, core="svmlight", kernel="rbf", C=1.0, 
                      #  gamma=0.5, example.weights=dm)

#summary(svm.wt)
#predicted <- predict(svm.wt,X.train)
#(predicted)

In [13]:
acc=c()
for (K in 1:k ){
    print(K)
    
    #split train test
X.train<-cv(K,fh)$X.train
y.train<-cv(K,fh)$y.train
X.test<-cv(K,fh)$X.test
y.test<-cv(K,fh)$y.test
    

    ###set max iterations T
T=2
#initialize sample weights
dm=replicate(nrow(X.train),1/nrow(X.train))
#length(dm)
    
###aplha parameter from adaboost algorith 
alpha=c()

###list to store predictions
pred=list()

###list to store test vector predictions
testpred=list()

###define number of classes
classes=11
#dm

###run adabosst multiclass with cost vector to each class
for (t in 1:T){
svm.wt <- SVM(X.train,y.train, core="svmlight", kernel="rbf", C=1.0, 
                        gamma=0.5, example.weights=dm)

#summary(svm.wt)
#predict test values
predicted_test <- predict(svm.wt,X.test)
#predict train values
predicted_train<- predict(svm.wt,X.train)    

    
#store predicted test values
testpred[[t]]<-(predicted_test)
    #print(testpred)
    
#store predicted train values
pred[[t]]<-(predicted_train)
    
#initially for each iteration error==0
err=0
    
    #for each example in train predictions check misclassification
    for (i in (1:nrow(X.train))){
        if (pred[[t]][i] != y.train[i]){
            y=y.train[i]
            err=err + dm[i] #* cost[y-1]
            }
        }
    #calculate alpha wth err value and classes
    alpha[t]<-(log((1-err)/err) + log(classes-1))
    
    #calculate Z value for normaliztion
    Z=c()
    #print(err)
    for (i in (1:nrow(X.train))){
        #print(dm)
        #print(as.numeric( predicted_train[i]))
        Z[i]<-(dm[i] * exp((-(alpha[t]))*(as.numeric(y.train[i])) *as.numeric( predicted_train[i])))
        #print(Z)
    }
    #update the weights
    
    for (i in (1:nrow(X.train))){
        dm[i]=Z[i] / sum(Z)
        #print(dm)
        }

        
}   

#initialize the adaboost ensemble otput for each class
ada_class=c()
ada_class_pred=list()
#run for loop for number of classes
for (c in (1:classes)){
    
    #initialize predictions for each test vector
    
    
    #for each test vector 
    #for (i in 1:nrow(testpred[t]) ){
        
        #initialize the adaboost output as zero for each class
        ada_predicts=replicate(length(testpred[[1]]),0)
        #calculate the adaboost predicted value across t generations.this is a scalar
        for (t in 1:T){
            vec=testpred[[t]]
            al=alpha[t]
           # print(vec)
            for (i in 1:length(vec)){
            if (vec[i]==c){
                ada_predicts[i]= ada_predicts[i]+(al *1)
                }
            else{
                ada_predicts[i]=ada_predicts[i]+(al *0)
                }
            }
        }
        
    ada_class_pred[[c]]<-(ada_predicts)
        
    #}
    #predictions of each test vector across classes
    #ada_class.append(ada_class_pred)
}
#ada_class_pred

x<-ada_class_pred
cl=c()
for (i in 1:length(testpred[[1]]))
{
m = as.numeric(do.call( rbind, x)[,i])
    #print(m)
cl[i]<-which(m==max(m))[1]
}
#

acc[K] = length(which(y.test == cl)) / length(cl)
    print(acc[K])

}



[1] 1


Warning message in SVM.default(X.train, y.train, core = "svmlight", kernel = "rbf", :
“It is recommended to pass y as factor”

In [ ]:
avg.acc=sum(acc)/k
avg.acc

In [ ]:
X.train<-cv(1,fh)$X.train
y.train<-cv(1,fh)$y.train
X.test<-cv(1,fh)$X.test
y.test<-cv(1,fh)$y.test
svm.wt <- SVM(X.train,y.train, kernel="rbf", C=1.0, 
                        gamma=0.5, example.weights=dm)

#summary(svm.wt)
#predict test values
predicted_test <- predict(svm.wt,X.test)
#predict train values
predicted_train<- predict(svm.wt,X.train) 

Warning message in SVM.default(X.train, y.train, kernel = "rbf", C = 1, gamma = 0.5, :
“It is recommended to pass y as factor”

optimization finished, #iter = 6222
optimization finished, #iter = 8694
optimization finished, #iter = 3810
optimization finished, #iter = 4930
optimization finished, #iter = 5912
optimization finished, #iter = 6305
optimization finished, #iter = 2956
optimization finished, #iter = 142
optimization finished, #iter = 1225
optimization finished, #iter = 502
optimization finished, #iter = 255


In [19]:
?SVM